# Taller Introductorio de Transformers:
#### Traducción y Clasificación de Sentimientos en Poemas

In [ ]:
# ejecutar esto en colab
pip install datasets
pip install accelerate -U

In [1]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report


## Objetivo
En este taller práctico, aplicaremos los conocimientos adquiridos sobre transformers para realizar una tarea completa de procesamiento de lenguaje natural. Comenzaremos con la traducción de poemas del inglés al español y luego realizaremos una clasificación de sentimientos utilizando modelos pre-entrenados en español.

## Requisitos previos
- Conocimientos básicos de Python y PyTorch
- Familiaridad con los conceptos de transformers y procesamiento de lenguaje natural
- Haber revisado el contenido de los scripts de introducción proporcionados

## Parte 1: Carga y Traducción del Dataset (1 hora)

### 1.1 Cargar el dataset de sentimientos de poemas desde Hugging Face

Cargue el dataset "google-research-datasets/poem_sentiment" utilizando la función `load_dataset` de la biblioteca `datasets`. Explore el contenido del dataset imprimiendo su información general y el primer elemento. Analice la estructura de los datos y discuta qué tipo de información contiene este dataset.


## Requisitos previos
- Conocimientos básicos de Python y PyTorch
- Familiaridad con los conceptos de transformers y procesamiento de lenguaje natural
- Haber revisado el contenido de los scripts de introducción proporcionados

## Parte 1: Carga y Traducción del Dataset (1 hora)

### 1.1 Cargar el dataset de sentimientos de poemas desde Hugging Face

Cargue el dataset `google-research-datasets/poem_sentiment` utilizando la función `load_dataset` de la biblioteca `datasets`. Explore el contenido del dataset imprimiendo su información general y el primer elemento. Analice la estructura de los datos y discuta qué tipo de información contiene este dataset.

In [ ]:
# Cargar el dataset
# COLOCAR AQUI EL CODIGO

In [ ]:

# Mostrar información del dataset
print(dataset)
print(dataset['train'][0])

# Analizar la distribución de sentimientos
sentiment_distribution = dataset['train'].map(lambda x: {'label': x['label']}, batched=True, batch_size=1000) # Return a dictionary
# Convert the 'label' column to a Pandas Series to use value_counts()
print(pd.Series(sentiment_distribution['label']).value_counts())


### 1.2 Traducir los poemas del inglés al español

Utilice el modelo de traducción "Helsinki-NLP/opus-mt-en-es" para traducir los poemas del dataset del inglés al español. Cree una función que traduzca un texto dado y aplíquela a todo el dataset utilizando el método `map`. Compare algunos ejemplos de poemas originales con sus traducciones y discuta la calidad de la traducción, considerando especialmente cómo se mantienen (o no) los elementos poéticos.


In [ ]:
import torch
from transformers import pipeline

# Crear el pipeline de traducción usando GPU si está disponible Y cargar el pipeline de translation en gpu
device = "cuda" if torch.cuda.is_available() else "cpu"
translator = # COLOCAR AQUI EL CODIGO

# Función para traducir un lote de textos
def translate_text(examples):
    translations = translator(examples['verse_text'], max_length=512, batch_size=32)
    return {"translated_poem": [t['translation_text'] for t in translations]}

# Aplicar la traducción al dataset
translated_dataset = dataset['train'].map(translate_text, batched=True, batch_size=32)


In [13]:
# Mostrar un ejemplo de poema original y traducido
print("Poema original:", dataset['train'][0]['verse_text'])
print("Poema traducido:", translated_dataset[0]['translated_poem'])
print("Sentimiento:", dataset['train'][0]['label'])

Poema original: with pale blue berries. in these peaceful shades--
Poema traducido: con bayas azules pálidas. en estos tonos pacíficos...
Sentimiento: 1


In [ ]:

# # Guardar el dataset traducido
# # translated_dataset.save_to_disk("./translated_poem_sentiment")

# print("Dataset traducido guardado en ./translated_poem_sentiment")

# # Para verificar, cargar un ejemplo del dataset guardado
# from datasets import load_from_disk

# translated_dataset = load_from_disk("./translated_poem_sentiment")
# print("Ejemplo del dataset guardado:")
# # print("Poema original:", dataset['train'][0]['verse_text'])
# print("Poema traducido:", translated_dataset[0]['translated_poem'])
# print("Sentimiento:", translated_dataset[0]['label'])

## Parte 2: Clasificación de Sentimientos en Poemas Traducidos (2 horas)

### 2.1 Preparar el dataset para clasificación

Prepare el dataset traducido para la tarea de clasificación de sentimientos. Asegúrese de que las etiquetas de sentimiento estén en un formato adecuado para el entrenamiento (por ejemplo, convertidas a valores numéricos). Analice la distribución de las etiquetas de sentimiento y discuta cómo esto podría afectar el entrenamiento de un modelo de clasificación.


In [ ]:
# Convertir etiquetas de sentimiento a valores numéricos, 
sentiment_map = # COLOCAR AQUI EL CODIGO # {"negative": 0, ...}

def prepare_for_classification(example):
    # Handle cases where 'label' is already an integer
    label_value = sentiment_map.get(example['label'], example['label'])
    return {
        "text": example['translated_poem'],
        "label": label_value
    }

classified_dataset = translated_dataset.map(prepare_for_classification)

# Analizar la distribución de etiquetas
label_distribution = classified_dataset.map(lambda x: {"label": x['label']})

# Convert the Dataset to a Pandas DataFrame to use value_counts()
# COLOCAR AQUI EL CODIGO

### 2.2 Extracción de características usando un modelo pre-entrenado en español

Utilice el modelo "dccuchile/bert-base-spanish-wwm-uncased" para extraer características de los poemas traducidos. Cree una función que tome un lote de textos, los tokenice y pase por el modelo para obtener los embeddings de la última capa oculta. Aplique esta función a todo el dataset y analice la forma y el contenido de las características extraídas.


In [ ]:
# Cargar el tokenizador y el modelo
tokenizer = # COLOCAR AQUI EL CODIGO
model = # COLOCAR AQUI EL CODIGO


In [ ]:

# Función para extraer características
def extract_features(batch):
    inputs = # COLOCAR AQUI EL CODIGO
    with torch.no_grad():
        outputs = model(**inputs)
    return {"features": outputs.last_hidden_state[:, 0, :].numpy()}

# Extraer características
dataset_with_features = # COLOCAR AQUI EL CODIGO con .. classified_dataset.map()


In [ ]:

# Analizar las características extraídas
print("Forma de las características:", np.array(dataset_with_features[0]['features']).shape)
print("Primeros 5 valores de las características:", dataset_with_features[0]['features'][:5])

### 2.3 Entrenar un clasificador simple usando las características extraídas

Utilice las características extraídas para entrenar un clasificador de regresión logística multiclase. Divida el dataset en conjuntos de entrenamiento y prueba. Entrene el modelo en el conjunto de entrenamiento y evalúe su rendimiento en el conjunto de prueba. Analice los resultados obtenidos, incluyendo la exactitud (accuracy), matriz de confusión y classification report. Compare estos resultados con un clasificador dummy que siempre predice la clase más frecuente.

In [ ]:
# Preparar los datos
X = dataset_with_features["features"]
y = dataset_with_features["label"]

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el clasificador
# COLOCAR AQUI EL CODIGO

# Evaluar el clasificador
# COLOCAR AQUI EL CODIGO


In [ ]:

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["negativo", "neutral", "positivo", "mixed"]))

In [ ]:
# Comparar con un clasificador dummy
dummy_clf = # COLOCAR AQUI EL CODIGO
dummy_clf.fit(X_train, y_train)
dummy_pred = # COLOCAR AQUI EL CODIGO
print("\nDummy Classifier Accuracy:", accuracy_score(y_test, dummy_pred))

### 2.4 Fine-tuning de un modelo pre-entrenado en español para clasificación de sentimientos

Utilice el modelo "dccuchile/bert-base-spanish-wwm-uncased" como base para realizar un fine-tuning para nuestra tarea de clasificación de sentimientos en poemas. Configure los hiperparámetros de entrenamiento, incluyendo el número de épocas, el tamaño del lote y la tasa de aprendizaje. Entrene el modelo utilizando la clase `Trainer` de Hugging Face. Evalúe el modelo fine-tuned en el conjunto de validación y compare sus resultados con los obtenidos en el paso anterior. Discuta las ventajas y desventajas de este enfoque en comparación con el uso de características extraídas.

In [ ]:
# Preparar el dataset para fine-tuning
def tokenize_function(examples):
    return # COLOCAR AQUI EL CODIGO with tokenizer(parametros)

tokenized_dataset = classified_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["id", "verse_text", "translated_poem"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset = tokenized_dataset.with_format("torch")

# Dividir el dataset
train_dataset = tokenized_dataset.shuffle(seed=42).select(range(int(len(tokenized_dataset)*0.8)))
eval_dataset = tokenized_dataset.shuffle(seed=42).select(range(int(len(tokenized_dataset)*0.8), len(tokenized_dataset)))

In [9]:
# Cargar el modelo para fine-tuning
model = # COLOCAR AQUI EL CODIGO

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Configurar el entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [11]:
# Crear el Trainer
# COLOCAR AQUI EL CODIGO

In [ ]:

# Entrenar el modelo


In [ ]:
# Evaluar el modelo
eval_results = trainer.evaluate()
print(eval_results)


In [ ]:

# Realizar predicciones con el modelo fine-tuned generando classification report (opcionalmente matriz de confusion)


## Conclusión y Discusión (15 minutos)

Reflexione sobre los resultados obtenidos en las diferentes etapas del taller. Compare los enfoques de extracción de características y fine-tuning, discutiendo sus ventajas y desventajas en términos de rendimiento, tiempo de entrenamiento y recursos computacionales requeridos. Considere cómo estos métodos podrían aplicarse a problemas del mundo real relacionados con el análisis de sentimientos en textos literarios y qué modificaciones podrían ser necesarias para mejorar aún más los resultados.

#### Preguntas para la discusión:
1. ¿Cómo afectó la calidad de la traducción a los resultados de la clasificación de sentimientos en los poemas?
2. ¿Qué desafíos específicos presenta la clasificación de sentimientos en poemas en comparación con textos más convencionales?
3. ¿Qué enfoque (extracción de características vs. fine-tuning) funcionó mejor para esta tarea y por qué cree que fue así?
4. ¿Cómo podríamos adaptar este modelo para analizar sentimientos en otros tipos de textos literarios, como cuentos cortos o letras de canciones?
5. ¿Qué implicaciones éticas y culturales debemos considerar al utilizar modelos de IA para analizar y clasificar obras literarias?

Este taller actualizado proporciona una experiencia práctica en el uso de transformers para tareas de procesamiento de lenguaje natural, enfocándose en la clasificación de sentimientos en poemas traducidos. Los estudiantes podrán aplicar los conocimientos adquiridos previamente, explorar nuevas técnicas en un contexto literario y reflexionar sobre las implicaciones de sus resultados en el análisis computacional de la literatura.


RESPONDER AQUI